In [ ]:
# encoding:utf-8
import requests
import json
import urllib
import urllib.request
import time
import struct
import pyaudio
import pvporcupine
import wave
import speech_recognition as sr
from aip import AipSpeech



APP_ID = '24690625'
API_KEY = '4wvprFyDikit7dGDmr3qMVW1'
SECRET_KEY = 'xGZ3AyezrZpncTYnTaXdoONVWfvKB3op'
client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)
url = 'https://aip.baidubce.com/rpc/2.0/unit/service/v3/chat?access_token=24.7a069087f696cb7d853f3ee33a4efcbf.2592000.1676606399.282335-29476996' # token id
headers = {'content-type': 'application/x-www-form-urlencoded'}
access_key = "c7E0vdPeiP6qg4pXo/AdHrMq9a6UFx6qczLYex7g0qKwmFkJm0UEmQ==" #porcupine
porcupine = pvporcupine.create( access_key=access_key,keywords=['porcupine', 'ok google', "picovoice", "blueberry"])
# porcupine = pvporcupine.create(access_key=access_key,keyword_paths=["graduation_en_raspberry-pi_v2_1_0.ppn"])
framerate=16000
NUM_SAMPLES=2000
channels=1
sampwidth=2
TIME=5

def save_wave_file(filename,data):
    '''save the data to the wavfile'''
    wf=wave.open(filename,'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(sampwidth)
    wf.setframerate(framerate)
    wf.writeframes(b"".join(data))
    wf.close()
# 直接调本函数即可录制时长为TIME的音频output.wav
def my_record():
    pa=pyaudio.PyAudio()
    stream=pa.open(format = pyaudio.paInt16,channels=1,
                   rate=framerate,input=True,
                   frames_per_buffer=NUM_SAMPLES)
    my_buf=[]
    count=0
    while count<TIME*8:#控制录音时间
        string_audio_data = stream.read(NUM_SAMPLES)
        my_buf.append(string_audio_data)
        count+=1
        print('.',end="")
    save_wave_file('output.wav',my_buf)
    stream.close()
# 直接调取麦克风权限录音5秒，返回识别结果
def voice_to_speech_google():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio_data = r.record(source, duration=5)
        print("Recognizing...")
        question = r.recognize_google(audio_data,language="zh-cn")
    return question
    print(question)
# 输出音频识别结果
def voice_to_text_baidu_sdk(filePath):
    with open(filePath, 'rb') as fp:
    # 识别本地文件
        start = time.time()
        result = client.asr(fp.read(), 'pcm', 16000, {
            'dev_pid': 1537,
        })
        end = time.time()
        return(result.get('result')[0])
        # print(end-start)
        # print(result.get('result')[0])

# 输入为字符串，输出为问题答案
def chat_unit(QUESTION):   
    post_data = "{\"version\":\"3.0\",\"service_id\":\"S80122\",\"session_id\":\"\",\"log_id\":\"1114749\",\"request\":{\"terminal_id\":\"88888\",\"query\":\"" + QUESTION +"\"}} "
    response = requests.post(url, data=post_data.encode('utf-8'), headers=headers)
    answer = json.loads(response.text)
    return answer
    # print(answer['result']['responses'][0]['actions'][0]['say'])


pa = pyaudio.PyAudio()
audio_stream = pa.open(
                    rate=porcupine.sample_rate,
                    channels=1,
                    format=pyaudio.paInt16,
                    input=True,
                    frames_per_buffer=porcupine.frame_length)

while True:
    pcm = audio_stream.read(porcupine.frame_length)
    pcm = struct.unpack_from("h" * porcupine.frame_length, pcm)
    keyword_index = porcupine.process(pcm)
    if keyword_index >= 0:
        # Insert detection event callback here
        print("收到,请讲话：")
        my_record()
        question = voice_to_text_baidu_sdk('output.wav')

        # print(chat_unit(question)['result']['context']['SYS_PRESUMED_HIST'][-1])
        print(chat_unit(question)['result']['context']['SYS_PRESUMED_HIST'][-1])


In [20]:
# 极速版,精简版
# coding=utf-8

import sys
import json
import base64
import time
import os
from urllib.request import urlopen
from urllib.request import Request
from urllib.error import URLError
from urllib.parse import urlencode
# 需要识别的文件
AUDIO_FILE = 'intent_repeat.wav'  # 只支持 pcm/wav/amr 格式，极速版额外支持m4a 格式
# 文件格式
FORMAT = AUDIO_FILE[-3:]  # 文件后缀只支持 pcm/wav/amr 格式，极速版额外支持m4a 格式

DEV_PID = 80001
ASR_URL = 'http://vop.baidu.com/pro_api'
SCOPE = 'brain_enhanced_asr'  # 有此scope表示有极速版能力，没有请在网页里开通极速版
token = '24.bfe537ccc417e898656cb29c37651a2c.2592000.1677248174.282335-29945070'
def voice_to_text_baidu_jisu():
    length = os.path.getsize(AUDIO_FILE)
    file1 = open(AUDIO_FILE, "rb").read()   # 读取本地语音文件   
    speech = base64.b64encode(file1).decode("utf-8")
    params = {'dev_pid': DEV_PID,
             #"lm_id" : LM_ID,    #测试自训练平台开启此项
              'format': FORMAT,
              'rate': 16000,
              'token': token,
              'cuid': "DC-85-DE-F9-08-59",
              'channel': 1,
              'speech': speech,
              'len': length
              }
    post_data = json.dumps(params, sort_keys=False)
    req = Request(ASR_URL, post_data.encode('utf-8'))
    req.add_header('Content-Type', 'application/json')
    try:
        f = urlopen(req)
        result_str = f.read()
    except URLError as err:
        print('asr http response http code : ' + str(err.code))
        result_str = err.read()

    result_str = str(result_str, 'utf-8')
    print(result_str)

In [4]:
chat_unit('枣庄今天天气怎么样')['result']['context']['SYS_PRESUMED_HIST'][-1]
# voice_to_speech_google()
# voice_to_text_baidu_jisu()
# voice_to_text_baidu_sdk('01.wav')

'枣庄今天晴转多云气温零下2到零上12度~'

In [ ]:
import pygame
def speak(name):
    # frequency用来控制语速，去掉试一下就明白了。。。
    pygame.mixer.init(frequency=16000, size=0)
    pygame.mixer.music.load(name)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        continue
    pygame.mixer.music.stop()

In [ ]:
# 识别到关键字后输出操作（后续开发思路）
import struct
import pyaudio
import pvporcupine
access_key = "c7E0vdPeiP6qg4pXo/AdHrMq9a6UFx6qczLYex7g0qKwmFkJm0UEmQ=="
porcupine = pvporcupine.create( access_key=access_key,keywords=['porcupine', 'ok google', "picovoice", "blueberry"])
# porcupine = pvporcupine.create(keyword_paths=['graduation_en_raspberry-pi_v2_1_0.ppn'])
pa = pyaudio.PyAudio()
audio_stream = pa.open(
                    rate=porcupine.sample_rate,
                    channels=1,
                    format=pyaudio.paInt16,
                    input=True,
                    frames_per_buffer=porcupine.frame_length)
#def get_next_audio_frame():
#  return 
while True:
    pcm = audio_stream.read(porcupine.frame_length)
    pcm = struct.unpack_from("h" * porcupine.frame_length, pcm)
    keyword_index = porcupine.process(pcm)
    if keyword_index >= 0:
        # Insert detection event callback here
        print("收到")
        question = voice_to_speech_baidu_sdk()
        


In [40]:
import wave
import pyaudio 
framerate=16000
NUM_SAMPLES=2000
channels=1
sampwidth=2
TIME=5

def save_wave_file(filename,data):
    '''save the data to the wavfile'''
    wf=wave.open(filename,'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(sampwidth)
    wf.setframerate(framerate)
    wf.writeframes(b"".join(data))
    wf.close()
def my_record():
    pa=pyaudio.PyAudio()
    stream=pa.open(format = pyaudio.paInt16,channels=1,
                   rate=framerate,input=True,
                   frames_per_buffer=NUM_SAMPLES)
    my_buf=[]
    count=0
    while count<TIME*8:#控制录音时间
        string_audio_data = stream.read(NUM_SAMPLES)
        my_buf.append(string_audio_data)
        count+=1
        print('.',end="")
    save_wave_file('01.wav',my_buf)
    stream.close()

In [ ]:
import pvcobra
import struct
import pyaudio
import wave

access_key = "c7E0vdPeiP6qg4pXo/AdHrMq9a6UFx6qczLYex7g0qKwmFkJm0UEmQ==" #porcupine
cobra = pvcobra.create(access_key=access_key)

def save_wave_file(filename,data):
    '''save the data to the wavfile'''
    wf=wave.open(filename,'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(sampwidth)
    wf.setframerate(framerate)
    wf.writeframes(b"".join(data))
    wf.close()

pa = pyaudio.PyAudio()
audio_stream = pa.open(
                    rate=porcupine.sample_rate,
                    channels=1,
                    format=pyaudio.paInt16,
                    input=True,
                    frames_per_buffer=porcupine.frame_length)


while True:
    pcm = audio_stream.read(512)
    pcm = struct.unpack_from("h" * 512, pcm) #if delete:Invalid frame length. expected 512 but received 1024
    # keyword_index = porcupine.process(pcm)
    voice_probability = cobra.process(pcm)

    if cobra.process(pcm) >= 0.8:
        print("在说话")
        record_buf = []
        while cobra.process(pcm) > 0.3:
            print(cobra.process(pcm))
            record_buf.append(pcm)
        print('说话结束')
        save_wave_file('test.wav',record_buf)    
        break


In [38]:
import pvcobra
import struct
import pyaudio
import wave

access_key = "c7E0vdPeiP6qg4pXo/AdHrMq9a6UFx6qczLYex7g0qKwmFkJm0UEmQ==" #porcupine
cobra = pvcobra.create(access_key=access_key)

def save_wave_file(filename,data):
    '''save the data to the wavfile'''
    wf=wave.open(filename,'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(sampwidth)
    wf.setframerate(framerate)
    wf.writeframes(b"".join(data))
    wf.close()

pa = pyaudio.PyAudio()
audio_stream = pa.open(
                    rate=porcupine.sample_rate,
                    channels=1,
                    format=pyaudio.paInt16,
                    input=True,
                    frames_per_buffer=porcupine.frame_length)
status = 1

while True:
    pcm = audio_stream.read(512)
    pcm = struct.unpack_from("h" * 512, pcm) #if delete:Invalid frame length. expected 512 but received 1024
    # keyword_index = porcupine.process(pcm)
    voice_probability = cobra.process(pcm)
    print(cobra.process(pcm)) 
    record_buf = []
    while cobra.process(pcm) >= 0.8:
        print("在说话")
        record_buf.append(pcm)
        # while cobra.process(pcm) > 0.3:
        #     print('ok')
        #     print(cobra.process(pcm))
        #     record_buf.append(pcm)
        if cobra.process(pcm) < 0.3:
            save_wave_file('test.wav',record_buf)
            print('说话结束')
            status = 0
            break
    if status == 0:
        break

0.011331753805279732
0.018440209329128265
0.017309095710515976
0.025035424157977104
0.013821961358189583
0.011574994772672653
0.019775839522480965
0.011732464656233788
0.010101075284183025
0.011061472818255424
0.01327388733625412
0.009984569624066353
0.00976497121155262
0.010683431290090084
0.00968971662223339
0.010418377816677094
0.009869393892586231
0.010278362780809402
0.009812300093472004
0.00973668321967125
0.011915022507309914
0.009869393892586231
0.00982179306447506
0.011104285717010498
0.009955651126801968
0.010519541800022125
0.010159831494092941
0.010238700546324253
0.012088717892765999
0.009812300093472004
0.009029013104736805
0.008968053385615349
0.010388216003775597
0.009161027148365974
0.009285964071750641
0.009205454960465431
0.009605741128325462
0.00982179306447506
0.009143315255641937
0.008933402597904205
0.009550157934427261
0.009974921122193336
0.009531700983643532
0.009439945220947266
0.010110843926668167
0.010179492644965649
0.009513279423117638
0.01015001535415649

KeyboardInterrupt: 

In [35]:
# 识别到关键字后输出操作（后续开发思路）
import struct
import pyaudio
import pvporcupine
import pvcobra
access_key = "c7E0vdPeiP6qg4pXo/AdHrMq9a6UFx6qczLYex7g0qKwmFkJm0UEmQ=="
porcupine = pvporcupine.create( access_key=access_key,keywords=['porcupine', 'ok google', "picovoice", "blueberry"])
cobra = pvcobra.create(access_key=access_key)

# porcupine = pvporcupine.create(access_key=access_key,keyword_paths=['C:/Users/ASUS/Documents/GitHub/bishe/voice_develop/all_devlop/graduation_en_raspberry-pi_v2_1_0.ppn'],sensitivities=[0.7])
pa = pyaudio.PyAudio()
audio_stream = pa.open(
                    rate=porcupine.sample_rate,
                    channels=1,
                    format=pyaudio.paInt16,
                    input=True,
                    frames_per_buffer=porcupine.frame_length)
#def get_next_audio_frame():
#  return 
while True:
    pcm = audio_stream.read(porcupine.frame_length)
    pcm = struct.unpack_from("h" * porcupine.frame_length, pcm)
    voice_probability = cobra.process(pcm)
    print(voice_probability)
            
                
           

0.029873091727495193
0.029535340145230293
0.026105914264917374
0.013402404263615608
0.020508162677288055
0.011125752702355385
0.009055264294147491
0.027040470391511917
0.02598205953836441
0.02931223064661026
0.01971912756562233
0.0193265862762928
0.013808656483888626
0.014406194910407066
0.012467670254409313
0.016293104737997055
0.011846236884593964
0.011331753805279732
0.012253088876605034
0.021409770473837852
0.033494532108306885
0.06412401050329208
0.7150261998176575
0.966945230960846
0.9857180714607239
0.9828068017959595
0.9765129685401917
0.9840323328971863
0.9798420071601868
0.9899086952209473
0.9862312078475952
0.986084520816803
0.9888095855712891
0.9750597476959229
0.9924300909042358
0.9901019334793091
0.9921385645866394
0.992774248123169
0.992038905620575
0.9922595024108887
0.9927321076393127
0.9933781623840332
0.9936733841896057
0.9931760430335999
0.993492841720581
0.08359067142009735
0.9854964017868042
0.9822707176208496
0.9907588362693787
0.9918674230575562
0.99202346801757

KeyboardInterrupt: 